<a href="https://colab.research.google.com/github/voldrek/Meus_projetos/blob/main/RomaneioVersionDefinitiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re,os,openpyxl,chardet
import numpy as np
from openpyxl import load_workbook

# Pasta onde os arquivos CSV estão localizados
pasta_arquivos = 'teste'

# Lista para armazenar os DataFrames de cada arquivo CSV
dataframes = []

#Função para encontrar o código utf das pastas
def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        result = chardet.detect(file.read())
    return result['encoding']



# Iterar pelos arquivos na pasta
for arquivo in os.listdir(pasta_arquivos):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(pasta_arquivos, arquivo)
        df = pd.read_csv(caminho_arquivo, sep='\t', encoding='utf-16')
        dataframes.append(df)

# Concatenar os DataFrames em um único DataFrame
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Salvar o DataFrame concatenado em um novo arquivo CSV
arquivo_concatenado = 'concatenado.csv'
df_concatenado.to_csv(arquivo_concatenado, index=False, sep='\t', encoding='utf-16')

file_path = arquivo_concatenado
encoding = detect_encoding(file_path)
# Ler os dados do arquivo CSV
df = pd.read_csv(arquivo_concatenado, sep="\t", encoding=encoding)
#Deletar
df = df[~df['Ação'].str.contains('Excluir')]
df = df[~df['Produto'].str.contains('Produto')]

#Trasnforma a coluna do Service Id em string
df['Service Id'] = df['Service Id'].apply(str)

# Variáveis para padrões de números para extração
padrao_celular = r'(\b\d{11}\b)'
#padrao_iccd = r'\b\d{20}\b'
padrao_imei = r'(\b\d{15}\b)'
padrao_nf = r'(\b\d{8}\b)'

#Tratamento dos dados da Coluna dos produtos selecionando apenas as linhas realcionadas ao IMEI
aparelhos_resultado = df[df['Service Id'].str.contains(padrao_imei, na=False)]
aparalhos_encontrados = aparelhos_resultado['Produto'].dropna()
df['Aparelhos'] = aparalhos_encontrados.dropna()


#Tratamento dos dados da Coluna da Ordem de venda selecionando apenas as linhas realcionadas ao IMEI
ordem_vendas_resultado = df[df['Service Id'].str.contains(padrao_imei, na=False)]
ordem_vendas_encontrados = ordem_vendas_resultado['Ordem de vendas'].dropna()
df['OV'] = ordem_vendas_encontrados.dropna()


#Tratamento dos dados da Coluna da Nota Fiscal selecionando apenas as linhas realcionadas ao IMEI
nota_fiscal_resultado = df[df['Service Id'].str.contains(padrao_imei, na=False)]
nota_fiscal_encontrados = nota_fiscal_resultado['Nota Fiscal'].dropna()
df['NF'] = nota_fiscal_encontrados.dropna()


#Tratamento dos dados da Coluna da Conta de serviço selecionando apenas as linhas realcionadas ao IMEI
conta_servico_resultado = df[df['Service Id'].str.contains(padrao_imei, na=False)]
conta_servico_encontrados = conta_servico_resultado['Conta de serviço'].dropna()
df['Conta'] = conta_servico_encontrados.dropna()

#Tratamento dos dados da Coluna da Conta de serviço selecionando apenas as linhas realcionadas ao IMEI
iccd_resultado = df[df['Tipo de serviço'].str.contains('Simcard', na=False)]
iccd_encontrados = iccd_resultado['Service Id'].dropna()
df['ICCD'] = iccd_encontrados.dropna()


# Fazendo a Separação do que são imei,numero de telefone e ICCD da coluna do service Id
for index, row in df.iterrows():
    service_data = row['Service Id']
    telefone = re.search(padrao_celular, service_data)
    imei = re.search(padrao_imei, service_data)
    if telefone:
        df.at[index, 'Numero telefone'] = telefone.group(0)
    if imei:
        if 'IMEI' in df.columns:
            df.at[index, 'IMEI'] = imei.group(0)
        else:
            df['IMEI'] = None


#Salva os arquivos na nova pasta caso eles sejam localizados
with pd.ExcelWriter('resposta_romaneio.xlsx') as writer:
    df2 = pd.DataFrame()
    df2 = df2.astype(str)

    if 'ICCD' in df.columns:
      numero_iccd = df['ICCD'].dropna()
      numero_iccd.to_excel(writer, sheet_name='Planilha1', startcol=0, index=False)  # Coluna 'ICCD' na coluna A
    if 'Numero telefone' in df.columns:
      numero_telefone = df['Numero telefone'].dropna()
      numero_telefone.to_excel(writer, sheet_name='Planilha1', startcol=1, index=False)  # Coluna 'Telefone' na coluna b
    if 'OV' in df.columns:
      numero_ordem = df['OV'].dropna()
      numero_ordem.to_excel(writer, sheet_name='Planilha1', startcol=2, index=False)  # Coluna 'Ordem de Venda' na coluna c
    if 'NF' in df.columns:
      numero_nf = df['NF'].dropna()
      numero_nf.to_excel(writer, sheet_name='Planilha1', startcol=3, index=False) # Coluna 'Nota Fiscal' na coluna c
    if 'IMEI' in df.columns:
      numero_imei = df['IMEI'].dropna()
      numero_imei.to_excel(writer, sheet_name='Planilha1', startcol=4, index=False)  # Coluna 'IMEI' na coluna E
    if 'Conta' in df.columns:
      numero_conta = df['Conta'].dropna()
      numero_conta.to_excel(writer, sheet_name='Planilha1', startcol=5, index=False) #Coluna 'Conta de Serviço' na coluna F
    if 'Aparelhos' in df.columns:
      aparelhos = df['Aparelhos'].dropna()
      aparelhos.to_excel(writer,sheet_name='Planilha1', startcol=6, index=False) #Coluna 'Aparelhos' na coluna G



#Passa uma função for em todas as colunas para elas não aparecerem como numeros cientificos
df2 = pd.read_excel('resposta_romaneio.xlsx')
colunas_inteiro = ['Numero telefone', 'OV', 'NF', 'IMEI', 'Conta']
for coluna in colunas_inteiro:
    df2[coluna] = df2[coluna].astype(float).apply(lambda x: '{:.0f}'.format(x) if not pd.isnull(x) else x)
df2.to_excel('resposta_romaneio.xlsx', index=False)

In [ ]:
df2['ICCD'].head()

In [ ]:
df2.info()

In [ ]:
df2